In [6]:
# import warnings
# warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rasterio as rio

import shapely as shp
import matplotlib as mpl
import matplotlib.pyplot as plt

from pathlib import Path as P


# import ee
# import geemap
# geemap.set_proxy(port=7890)
# ee.Authenticate()
# ee.Initialize(project='ee-shy')


In [7]:
import torch
import ast
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# coffee distibution prediction with geemap

In [ ]:

# model parameters
pars_feas = [
    [[10,5,50,20], [10,12,12,4]]
]
# [10,5,50,20] -> [10 bands from Sentinel-2, 5 time periods, 50 inputs features, 20 outputs]
# [10,12,12,4] -> full connected network structure for Sentinel bands of each time period 


par_merge = [[22,0], [22,16,8,4,1]]
# [22,0] -> 22 inputs from the outputs of formal feature preprocessing network
# [22,16,8,4,1] -> network structure of feature combination

class DNN(nn.Module):
    # define model parameters
    def __init__(self, pars_feas=None, par_merge=None):  
        super(DNN, self).__init__()

        self.pars_feas = pars_feas
        self.par_merge = par_merge
        self.relu = nn.ReLU()

        # mdl list of all features are combined into one, and each feature is connected by a full connected layer to form an mdl list
        list_mdl = []
        # pars_fea [10,5,50,20], par_mdl [10,16,8,4]
        for pars_fea,par_mdl in pars_feas:
            list_mdl_fea = []  # used to store lyr of multiple layer
            for t_idx in range(pars_fea[1]):
                list_lyr = []
                for i in range(len(par_mdl)-1):
                    list_lyr.extend([nn.Linear(par_mdl[i],par_mdl[i+1]), self.relu])
                list_mdl_fea.append(nn.Sequential(*list_lyr))
            list_mdl.append(nn.ModuleList(list_mdl_fea))
        self.fcs_fea = nn.ModuleList(list_mdl)

        par_lyrs = par_merge[1]
        list_lyr = []
        for i in range(len(par_lyrs)-2):
            list_lyr.extend([nn.Linear(par_lyrs[i],par_lyrs[i+1]), self.relu])
        list_lyr.append(nn.Linear(par_lyrs[-2],par_lyrs[-1]))
        self.fcs_end = nn.Sequential(*list_lyr)


    # forward propagation process
    def forward(self, x):
        offset = 0
        xs = []

        # cycle a few times for certain types of features e.g. S2 and topology
        for i in range(len(self.pars_feas)):
            fea_pars = self.pars_feas[i][0]
            # cycle each time period
            for t_idx in range(fea_pars[1]):
                
                x_t = x[:, offset+t_idx*fea_pars[0]: offset+(t_idx+1)*fea_pars[0]]
                # print(t_idx, 'x_t', x_t.shape)
                xs.append(self.fcs_fea[i][t_idx](x_t))
            offset += fea_pars[2]

        # for x in xs:
        #     print(x.shape)
        if self.par_merge[0][1]:
            x = torch.cat(xs+[x[:,offset:]], dim=1)
        else:
            x = torch.cat(xs, dim=1)  
        # print(x.shape, self.fcs_end)
        x = self.fcs_end(x)

        # if it is binary classification, use sigmoid function to convert to binary result
        if self.par_merge[-1][-1] == 1:
            x = nn.Sigmoid()(x)
            x = x.squeeze(-1)
            
        return x



In [ ]:
path_mdl_save = 'mdl_pars.pth'

# torch.save(mdl, path_mdl_save)

mdl = torch.load(path_mdl_save)


pars = {}
pars_ee = {}
for par_name, par_val in mdl.named_parameters():
    pars[par_name] = par_val.detach().cpu().numpy().tolist()
    pars_ee[par_name] = ee.Image(ee.Array(pars[par_name]))
    if 'bias' in par_name:
        pars_ee[par_name] = pars_ee[par_name].toArray(1)
    # print(par_name, par_val.shape)



## image preparation

In [ ]:

bands_s2 = ['blue', 'green', 'red', 're1','re2', 're3', 'nir','re4','swir1', 'swir2']
def imgs_s2(roi, date_start, date_end, cloud_p=40):
    return indexJoin(
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(roi)
        .filterDate(date_start, date_end).map(maskS2clouds), 
        ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY'),  'cloud_probability'
    ).map(lambda image: image.updateMask(image.select('probability').lte(cloud_p))).select(bands_s2)

# maska clouds and rename band
def maskS2clouds(image):
    qa = image.select('QA60') 
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0)).Or(image.select('B2').lte(2000))
    return (image.updateMask(mask).select([
        'B2','B3','B4','B5','B6', 'B7', 'B8', 'B8A', 'B11','B12'
    ], bands_s2).divide(10000)
    .copyProperties(image, ["system:time_start"]).copyProperties(image))

# Merge the S2 cloud probability dataset with the S2 imagery dataset and use the probability file to create a cloud mask on the S2 image
def indexJoin(collectionA, collectionB, propertyName):
    joined = ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(**{
        'primary': collectionA, 'secondary': collectionB,
        'condition': ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})
    }))
    return joined.map(lambda image: image.addBands(ee.Image(image.get(propertyName))))




In [6]:
band_s2 = 'blue,green,red,re1,re2,re3,nir,re4,swir1,swir2'.split(',')
col_s2 = bands_s2 + [f'{b}_{i}' for i in range(1,5) for b in bands_s2]

col_s2_band = band_s2 + [f'{b}_{i}' for i in range(1, 5) for b in band_s2]
col_s2_band = ['s2_'+c for c in col_s2_band]

col_topo = ['elevation', 'slope']  # , 'aspect'

cols = col_s2_band + col_topo

In [7]:
# img for pred

# mdl pars
snic_size = 8

# data pars

yr_pred = 2022
study_area = 'sp_buf_3km'
scale = 10
city = 'cf_5city'   

roi_pred = ee.FeatureCollection("projects/ee-shy/assets/coffee/citys/"+city)
ct_5city = ee.FeatureCollection("projects/ee-shy/assets/coffee/cts/ct_5city") 
  
def str_pre(img, pre):
  return img.rename(img.bandNames().map( 
    lambda i: ee.String(pre).cat(ee.String(i))
  )) 

# flex vars, different time period composite

# periods of all satellite 
date_pred_start = ee.Date.fromYMD(yr_pred,4,1)
date_pred_end = ee.Date.fromYMD(yr_pred+1,4,1) 
t_period_pred = ee.List([  # 1 8 9 11
  date_pred_start, date_pred_start.advance(7,'month'), date_pred_start.advance(8,'month'), 
  date_pred_start.advance(10,'month'), date_pred_start.advance(11,'month'), date_pred_end
]) 

img_pred = ee.Image([0])  

# cal of s2 features
imgs = imgs_s2(roi_pred, date_pred_start, date_pred_end) 

for i in range(t_period_pred.length().getInfo()-1):

  img = imgs.filterDate(t_period_pred.get(i), t_period_pred.get(i+1)).median().clip(roi_pred) 
  # Map.addLayer(img, vis_rgb, 'S_'+i, false) 
  img_s2_feas = str_pre(cal_S_vi(img), 's2_')   # cal_S_vi(img) img
  img_pred = img_pred.addBands(img_s2_feas)   # s2 features  VIs
  # img_pred = img_pred.addBands(img)   # original s2 bands

# Map.addLayer(img, vis_rgb, 'img pred', false) 


snic = ee.Algorithms.Image.Segmentation.SNIC(
  image=img_pred.select(col_s2_band), size=snic_size, compactness=.1, connectivity=8, neighborhoodSize=64,
)
# print('snic', snic)
clusters = snic.select('clusters')
# Map.addLayer(clusters.randomVisualizer(), {}, 'clusters pred')

img_pred = snic.rename([n.replace('_mean', '') for n in snic.bandNames().getInfo()])  # .select(sel_feas)



dem = ee.Image('USGS/SRTMGL1_003').clip(roi_pred).float() 
img_pred = img_pred.addBands([dem, ee.Terrain.slope(dem), ee.Terrain.aspect(dem)])  # .select(sel_feas)  





## predict coffee distribution

In [ ]:
# col_topo = ['elevation', 'slope']  # , 'aspect'

# prediction  

img_todo = img_pred
roi_todo = roi_pred

# fcs_fea.0/1.0~4.0/2/4/6.weight
# fcs_fea.0.0.0.bias

def cat_img(imgs):
    img = imgs[0]
    for i in range(1, len(imgs)):
        img = img.arrayCat(imgs[i], axis=0)
    return img

imgs_fea = []
offset = 0

imgs_sub = []
for idx_fea in range(len(pars_feas)):
    pars_fea, par_mdl = pars_feas[idx_fea]
    n_bands, n_ts, offset_fea, n_fea_out = pars_fea

    for t_idx in range(n_ts):
        # print(cols[offset+t_idx*n_bands: offset+(t_idx+1)*n_bands])
        
        img_t = img_todo.select(cols[offset+t_idx*n_bands: offset+(t_idx+1)*n_bands]).toArray().toArray(1)
        # Map.addLayer(img_t.arrayLengths().arrayFlatten([['a','b']]), {}, f'{k}, {t_idx}')
        # if idx_fea:
        #     print('idx_fea', idx_fea, 't', t_idx, 'dim', img_t.arrayDimensions().getInfo())
        for lyr_idx in range(len(par_mdl)-1):
            # print(f'k: {k}, t: {t_idx}, lyr: {lyr_idx}', pars[f'fcs_{k}.{t_idx}.{lyr_idx*2}.weight'].shape)
            w = pars_ee[f'fcs_fea.{idx_fea}.{t_idx}.{lyr_idx*2}.weight']
            b = pars_ee[f'fcs_fea.{idx_fea}.{t_idx}.{lyr_idx*2}.bias']
            img_t = w.matrixMultiply(img_t).add(b)
            img_t = img_t.gt(0).multiply(img_t)
        imgs_sub.append(img_t)
    
    offset += offset_fea
    

if par_merge[0][1]:
    imgs_sub.append(img_todo.select(cols[offset: ]).toArray().toArray(1))

img = cat_img(imgs_sub)

# fcs_end.0/2/4/6.weight 
# fcs_end.0.bias

n_lyr_end = len(par_merge[1])-1
for fc_idx in range(n_lyr_end):
    img = pars_ee[f'fcs_end.{fc_idx*2}.weight'].matrixMultiply(img).add(pars_ee[f'fcs_end.{fc_idx*2}.bias'])

    if fc_idx == n_lyr_end-1:
        if par_merge[-1][-1] == 1:
            # binary classification
            img = ee.Image(1).divide(img.arrayGet([0,0]).clip(roi_todo).multiply(-1).exp().add(ee.Image(1))).clip(roi_todo)
        else:
            # multi classification
            exp_arr = img.exp()
            sum_exp = exp_arr.arrayReduce(ee.Reducer.sum(), [0]).arrayRepeat(0, par_merge[-1][-1])
            img = exp_arr.divide(sum_exp).arrayProject([0]).arrayArgmax().arrayFlatten([['class']])
        # print('fc_idx', fc_idx, 'dim', img.arrayDimensions().getInfo())
    else:
        img = img.gt(0).multiply(img)

if par_merge[-1][-1] == 1:
    img_res = img.gt(0.5).selfMask()
else:
    img_res = img



In [9]:
# visualize result

palette = ['FF0000', 'e8beff', 'aaff00', '38a800', 'ffffbe', 'e1e1e1', '73b2ff']
names = ['coffee', 'cropland', 'sparse_veg', 'forest', 'bare', 'build', 'water']

Map = geemap.Map()
Map.addLayer(img_res, {'min': 1, 'max': 7, 'palette': palette}, 'res')  #  pixel
Map.centerObject(roi_pred)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# expor result
fn = '2022_yunnan_coffee_10m'
print(fn)  
task = ee.batch.Export.image.toDrive(
    image=img_res.int8(), region=roi_pred.geometry(), description=fn,
    folder='GEE_data', maxPixels=1e11, scale=30, crs='EPSG:4326'
)
task.start()

# DL model training with pytorch

In [ ]:

import sklearn as sk
# import sklearn.ensemble
# import sklearn.model_selection
# import sklearn.inspection
# import sklearn.datasets
import sklearn.metrics

## training data preparation

device(type='cuda', index=0)

In [ ]:


# snic_size = 8
yr_train = 2022


band_s2 = ['blue', 'green', 'red', 're1', 're2', 're3', 'nir', 're4', 'swir1', 'swir2']

df_train = gdf_train = gpd.read_file('data_train.shp')
df_test = gdf_test = gpd.read_file('data_test.shp')


# topographic feature
col_topo = ['elevation', 'slope']  # , 'aspect'

# features of Sentinel-2 frequency bands at different time periods
col_s2_band = band_s2 + [f'{b}_{i}' for i in range(1, 5) for b in band_s2]
col_s2_band = ['s2_'+c for c in col_s2_band]
cols = col_s2_band + col_topo

# column of catogory 
col_type = 'class_2typ' 

print(gdf_train.columns.to_list())

# remove null values
x_train, y_train = df_train.loc[:, cols], df_train[col_type]
idx = ~(x_train.isnull() | np.isinf(x_train)).any(axis=1)
x_train, y_train = x_train.loc[idx, ], y_train.loc[idx, ]

x_test, y_test = df_test.loc[:, cols], df_test[col_type]
idx = ~(x_test.isnull() | np.isinf(x_test)).any(axis=1)
x_test, y_test = x_test.loc[idx, ], y_test.loc[idx, ]

# Standardizing before random forest can improve data processing efficiency
# avg, std = x_train.mean(), x_train.std()
# x_train = (x_train - avg) / std

df_train.shape, x_train.shape, df_test.shape, x_test.shape

['s2_blue', 's2_green', 's2_red', 's2_re1', 's2_re2', 's2_re3', 's2_nir', 's2_re4', 's2_swir1', 's2_swir2', 's2_blue_1', 's2_green_1', 's2_red_1', 's2_re1_1', 's2_re2_1', 's2_re3_1', 's2_nir_1', 's2_re4_1', 's2_swir1_1', 's2_swir2_1', 's2_blue_2', 's2_green_2', 's2_red_2', 's2_re1_2', 's2_re2_2', 's2_re3_2', 's2_nir_2', 's2_re4_2', 's2_swir1_2', 's2_swir2_2', 's2_blue_3', 's2_green_3', 's2_red_3', 's2_re1_3', 's2_re2_3', 's2_re3_3', 's2_nir_3', 's2_re4_3', 's2_swir1_3', 's2_swir2_3', 's2_blue_4', 's2_green_4', 's2_red_4', 's2_re1_4', 's2_re2_4', 's2_re3_4', 's2_nir_4', 's2_re4_4', 's2_swir1_4', 's2_swir2_4', 'elevation', 'slope', 'class_2typ', 'geometry']


((85044, 54), (85044, 52), (25955, 54), (25955, 52))

In [13]:
# generate dataset

# col_topo = ['elevation', 'slope']  # , 'aspect'
torch.set_default_dtype(torch.float32)
# cols = col_s2_band + col_topo

train_x, test_x = df_train.loc[:, cols].values.astype(np.float32), df_test.loc[:, cols].values.astype(np.float32)
idx_mask = ~(np.isinf(train_x) | np.isnan(train_x)).any(axis=1)
idx_mask_test = ~(np.isinf(test_x) | np.isnan(test_x)).any(axis=1)

train_x, test_x = train_x[idx_mask, ], test_x[idx_mask_test]

# # multi classification
# col_type = 'class_dec'  
# train_y, test_y = df_train.loc[idx_mask, col_type]-1, df_test.loc[idx_mask_test, col_type]-1
# trainDataset = TensorDataset(torch.tensor(train_x, dtype=torch.float32), torch.tensor(train_y, dtype=torch.long))
# valDataset = TensorDataset(torch.tensor(test_x, dtype=torch.float32), torch.tensor(test_y ,dtype=torch.long))  # torch.int

# binary classification
col_type = 'class_2typ'  # origin 1 is cf, 2 is non-coffee, turn cf into 1, non-coffee into 0
train_y, test_y = 2-df_train.loc[idx_mask, col_type], 2-df_test.loc[idx_mask_test, col_type]
trainDataset = TensorDataset(torch.tensor(train_x, dtype=torch.float32), torch.tensor(train_y.values, dtype=torch.float32))
valDataset = TensorDataset(torch.tensor(test_x, dtype=torch.float32), torch.tensor(test_y.values, dtype=torch.float32))  # torch.int

batch_size = 256
trainLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
valLoader = DataLoader(valDataset, batch_size=batch_size, shuffle=False)

train_x.shape, test_x.shape

((85044, 52), (25955, 52))

## model structure

In [14]:

# model parameters
pars_feas = [
    [[10,5,50,20], [10,12,12,4]]
]
# [10,5,50,20] -> [10 bands from Sentinel-2, 5 time periods, 50 inputs features, 20 outputs]
# [10,12,12,4] -> full connected network structure for Sentinel bands of each time period 

par_merge = [[20,2], [22,16,16,8,7]]
# [22,0] -> 22 inputs from the outputs of formal feature preprocessing network
# [22,16,16,8,7] -> network structure of feature combination

class DNN(nn.Module):
    # define model parameters
    def __init__(self, pars_feas=None, par_merge=None):  
        super(DNN, self).__init__()

        self.pars_feas = pars_feas
        self.par_merge = par_merge
        self.relu = nn.ReLU()

        # mdl list of all features are combined into one, and each feature is connected by a full connected layer to form an mdl list
        list_mdl = []
        # pars_fea [10,5,50,20], par_mdl [10,16,8,4]
        for pars_fea,par_mdl in pars_feas:
            list_mdl_fea = []  # used to store lyr of multiple layer
            for t_idx in range(pars_fea[1]):
                list_lyr = []
                for i in range(len(par_mdl)-1):
                    list_lyr.extend([nn.Linear(par_mdl[i],par_mdl[i+1]), self.relu])
                list_mdl_fea.append(nn.Sequential(*list_lyr))
            list_mdl.append(nn.ModuleList(list_mdl_fea))
        self.fcs_fea = nn.ModuleList(list_mdl)

        par_lyrs = par_merge[1]
        list_lyr = []
        for i in range(len(par_lyrs)-2):
            list_lyr.extend([nn.Linear(par_lyrs[i],par_lyrs[i+1]), self.relu])
        list_lyr.append(nn.Linear(par_lyrs[-2],par_lyrs[-1]))
        self.fcs_end = nn.Sequential(*list_lyr)


    # forward propagation process
    def forward(self, x):
        offset = 0
        xs = []

        # cycle a few times for certain types of features e.g. S2 and topology
        for i in range(len(self.pars_feas)):
            fea_pars = self.pars_feas[i][0]
            # cycle each time period
            for t_idx in range(fea_pars[1]):
                
                x_t = x[:, offset+t_idx*fea_pars[0]: offset+(t_idx+1)*fea_pars[0]]
                # print(t_idx, 'x_t', x_t.shape)
                xs.append(self.fcs_fea[i][t_idx](x_t))
            offset += fea_pars[2]

        # for x in xs:
        #     print(x.shape)
        if self.par_merge[0][1]:
            x = torch.cat(xs+[x[:,offset:]], dim=1)
        else:
            x = torch.cat(xs, dim=1)  
        # print(x.shape, self.fcs_end)
        x = self.fcs_end(x)

        # if it is binary classification, use sigmoid function to convert to binary result
        if self.par_merge[-1][-1] == 1:
            x = nn.Sigmoid()(x)
            x = x.squeeze(-1)
            
        return x



In [15]:


if par_merge[-1][-1] == 1:
    criterion = nn.BCELoss()  # binary classification
else:
    criterion = nn.CrossEntropyLoss()  # multi classification


mdl = DNN(pars_feas=pars_feas, par_merge=par_merge).to(device)
optimizer = optim.Adam(mdl.parameters(), lr=0.001)  # , weight_decay=1e-5

## model training

In [ ]:
pd.DataFrame(assess).to_excel(f'result/mdl/DL_2022_{study_area_fn}_2yr.xlsx')  # data_path / 

In [ ]:
# model training

import warnings
warnings.filterwarnings("ignore")

num_epochs = 200

train_losses = []
train_accuracies = []
val_accuracies = []

assess = []
for epoch in range(num_epochs):

    mdl.train()

    train_loss_list, train_metric_list = [],[]

    for inputs, labels in trainLoader:
        # print('ipt_size', inputs.size())
        labels = labels.type(torch.LongTensor)
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = mdl(inputs)
        if par_merge[-1][-1] == 1:
            labels = labels.squeeze(-1)
        loss_i = criterion(outputs, labels)

        optimizer.zero_grad()
        loss_i.backward()
        optimizer.step()

        train_loss_list.append(loss_i.detach().cpu().numpy())

    loss = np.mean(train_loss_list)

    # Verify accuracy
    mdl.eval()

    outputs_list = []
    labels_list = []
    with torch.no_grad():
        for ipts, lbls in valLoader:
            inputs, labels = ipts.to(device), lbls.numpy()
            outputs = mdl(inputs)

            if par_merge[-1][-1] == 1:
                outputs_list.append(outputs.detach().cpu().numpy())
            else:
                _, predicted = torch.max(outputs.data, 1)
                predicted = predicted.detach().cpu().numpy()
                outputs_list.append(predicted)
            
            labels_list.append(labels)

    if par_merge[-1][-1] == 1:
        y_pred = np.concatenate(outputs_list) > 0.5
    else:
        y_pred = np.concatenate(outputs_list)
    y_test = np.concatenate(labels_list)

    s_report = sk.metrics.classification_report(y_test, y_pred)
    report_res = s_report.split()
    kp = sk.metrics.cohen_kappa_score(y_test, y_pred)
    assess.append([epoch, loss]+report_res[5:8]+[report_res[report_res.index('accuracy')+1], kp])

    if epoch%20 == 0:
        
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train loss: {loss.item():.4f}, acc:{assess[-1]}')

In [19]:
path_mdl_save = 'mdl_pars.pth'

# torch.save(mdl, path_mdl_save)
mdl = torch.load(path_mdl_save)
